SPECTRAL DESCRIPTORS AND FUNCTIONAL MAPS

In [1]:
import pyFM
from pyFM import mesh
from pyFM import functional
from pyFM import eval


from shape_utils.spectral_descr import calculate_descriptors
from shape_utils.utils import find_minimum_distance_meshes, save_data_to_csv, save_list_to_csv
from shape_utils.functional_maps import calculate_functional_maps
from shape_utils.similarity_scores import calculate_geodesic_norm_score
from analysis_tools.visualisation_correspondence import visualize_correspondence_meshplot, visualize_correspondence_aligned
from analysis_tools.superposition_shapes import calculate_rotation_translation_fixed
import os
import seaborn as sns
import numpy as np
import meshplot as mp
from scipy.linalg import norm
import matplotlib.pyplot as plt
import csv

from shape_utils.zernike_descr import get_inv
from shape_utils.meshes import remove_until_vertex
from shape_utils.similarity_scores import predict_similarity_zernike
import trimesh

Define functions to plot functional maps and spectral descriptors on 3D surfaces

In [8]:
def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

Calculate correspondace matrix, point to point map and disimilarity score for two input files with the surface meshes (.off/.obj)

In [ ]:

file_mesh1 = './example_data/well_conditioned_meshes/7V7F.obj'
file_mesh2 = './example_data/well_conditioned_meshes/7V7D.obj'


#load mesh with TriMesh and normalized area 
mesh1 = mesh.TriMesh(file_mesh1, area_normalize=True, center=False)
mesh2 = mesh.TriMesh(file_mesh2, area_normalize=True, center=False)

#Define model for functional map with pyFM
model = functional.FunctionalMapping(mesh1,mesh2) 

#Define crucial parameters for Spectral descriptors
neigvecs = 200      #No. of eigenvectors
ndescr = 100        #No. of descriptors
step = 1            #Step for including descriptors
n_ev = 50           #No. of eigen vectors to define functional maps
output = 'example_data/spectral_output_data/'   #output path for results 
descr = 'WKS'       #Type of shape descriptor (WKS- Wavelet Kernel Signatures)
landmarks = None    #Add landmarks if available

#Calculation of descriptors
descr1,descr2 = calculate_descriptors(model,neigvecs,n_ev,ndescr,step,landmarks,output,descr)
print(np.shape(descr1))

#Computation of point to point maps
p2p_21,FM = calculate_functional_maps(model,n_cpus=8,refine=None)


#Computation of disimilarity score 
sim_score = calculate_geodesic_norm_score(FM)
print('disimilarity score:', sim_score)


SAVE DESCRIPTORS AND FUNCTIONAL MAPS TO FILES

In [8]:
#set output directory and entry ids
output_dir = output
entry_id_1 = "7V7F"
entry_id_2 = "7V7D"

#save descriptors to file
wks_file_1 = os.path.join(output_dir,"{}_descr_{}.csv".format("WKS",entry_id_1))
wks_file_2 = os.path.join(output_dir,"{}_descr_{}.csv".format("WKS",entry_id_2))

if not os.path.exists(wks_file_1) or not os.path.exists(wks_file_2):
    data1 = np.array(descr1)
    data2 = np.array(descr2)
    save_data_to_csv(data1,wks_file_1)
    save_data_to_csv(data2,wks_file_2)

#Save functional map and point to point map

output_file_FM = os.path.join(output_dir,"{}_{}_FM.csv".format(entry_id_1,entry_id_2))
output_file_p2p21 = os.path.join(output_dir, "{}_{}_p2p21.csv".format(entry_id_1,entry_id_2))

if not os.path.exists(output_file_FM) or not os.path.exists(output_file_p2p21):
    save_data_to_csv(FM,output_file_FM)
    save_list_to_csv(p2p_21,output_file_p2p21)


Plot correspondace matrix and 3D maps

In [ ]:
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)
fig, (ax1) = plt.subplots(1, figsize=(3, 3))
sns.heatmap(FM,ax=ax1,vmax=1.0,vmin=-1.0,cmap='seismic')

Calculate disimilarity score and plot correspondace matrix from output file *_FM.csv

In [ ]:
file_FM ='./example_data/spectral_output_data/7V7F_7V7D_FM.csv'


with open(file_FM) as csvfile:
    FM = list(csv.reader(csvfile))
    FM = np.asarray(FM, dtype=float)

#calculate disimilarity score
sim_score = calculate_geodesic_norm_score(FM)
print('disimilarity score:', sim_score)

#plot correspondance matrix
fig, (ax1) = plt.subplots(1, figsize=(3, 3))
sns.heatmap(FM,ax=ax1,vmax=1.0,vmin=-1.0,cmap='seismic')

VISUALISE SURFACE DESCRIPTORS WITH ALIGNED ROTATION

In [ ]:
file_mesh1 = './example_data/well_conditioned_meshes/7V7F.obj'
file_mesh2 = './example_data/well_conditioned_meshes/7V7D.obj'
source_path=file_mesh1 
target_path=file_mesh2

map_path = './example_data/spectral_output_data/7V7F_7V7D_p2p21.csv'
out_gif = './example_data/spectral_output_data/spike_wks.gif'
R,T=calculate_rotation_translation_fixed(source_path,target_path, map_path)
t = T.flatten()
print("det(R) =", np.linalg.det(R))
visualize_correspondence_aligned(source_path, target_path, map_path, R,t)



ZERNIKE DESCRIPTORS

Calculate Zernike rotational invariant moments for surface meshes 

In [ ]:
#input surface meshes
file_mesh1 = './example_data/well_conditioned_meshes/6mka-A.obj'
file_mesh2 = './example_data/well_conditioned_meshes/6mkg-A.obj'

#ouput directory to save the Zernike moments 
output_dir = './example_data/zernike_output_data'

#Entry IDs
entry_id_1 = "6mka-A"
entry_id_2 = "6mkg-A"

#removing extra lines in .obj file (obj2grid has trouble to read extra header lines in .obj files)
remove_until_vertex(file_mesh1)
remove_until_vertex(file_mesh2)

#Computation of Zernike Moments
get_inv(file_mesh1,entry_id_1,'map2zernike', 'obj2grid',output_dir)
get_inv(file_mesh1,entry_id_2,'map2zernike', 'obj2grid',output_dir)

Compute descriptors and pair scores from a list of entries and meshes

In [ ]:

meshes_dir = './example_data/well_conditioned_meshes/' #directory containing the meshes
output_moments_dir = './example_data/zernike_output_data' #output directory to save Zernike moments files
output_scores_dir = './example_data/zernike_output_data' #directory to save scores file
file_entries='./example_data/well_conditioned_meshes/list_entries.txt' #list of entry ids to read files (mesh file named after entry id)

#read list of entries
with open(file_entries, 'r') as file:
    # Read all lines, stripping the newline character
    entry_list = [line.strip() for line in file]
print(np.shape(entry_list))

#read mesh files 
for entry in entry_list:
    mesh_file=os.path.join(meshes_dir,"{}.obj".format(entry))
    print(mesh_file)
    mesh = trimesh.load(mesh_file, file_type="obj")
    
    #removing extra lines in .obj file (obj2grid has trouble to read extra header lines in .obj files)
    remove_until_vertex(mesh_file)
    #compute Zernike moments for each mesh
    get_inv(mesh_file,entry,'map2zernike', 'obj2grid',output_dir)
    
#compute scores for the list of files and entries and save scores file in output_scores_dir
predict_similarity_zernike(output_moments_dir,output_scores_dir)